<a href="https://colab.research.google.com/github/AbhiSrvstv/Q-A-Chatbot/blob/main/Q_%26_A_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai pandas python-docx PyMuPDF --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 20.2 MB/s eta 0:00:00


In [4]:
# Step 1: Install required libraries
#!pip install groq pandas python-docx PyMuPDF --quiet

# Step 2: Import necessary modules
from google.colab import files
import pandas as pd
import fitz  # PyMuPDF
import docx
from groq import Groq

# Step 3: Configure Groq API
api_key = "gsk_OMKrTVzExk"
client = Groq(api_key=api_key)
model = "llama-3.1-8b-instant"

# Step 4: Upload a file
uploaded = files.upload()

# Step 5: Read the uploaded file based on its type
def read_file(filename):
    if filename.endswith('.txt'):
        with open(filename, 'r', encoding='utf-8') as f:
            return f.read()
    elif filename.endswith('.pdf'):
        doc = fitz.open(filename)
        text = ''
        for page in doc:
            text += page.get_text()
        return text
    elif filename.endswith('.csv'):
        df = pd.read_csv(filename)
        return df.to_string(index=False)
    elif filename.endswith('.docx'):
        doc = docx.Document(filename)
        return '\n'.join([para.text for para in doc.paragraphs])
    else:
        return "Unsupported file format."

file_content = ""
for fname in uploaded.keys():
    file_content = read_file(fname)

# Step 6: Initialize chat history
messages = [
    {
        "role": "system",
        "content": "You are a helpful assistant. Use the uploaded document to answer questions."
    },
    {
        "role": "user",
        "content": f"Here is the document content:\n\n{file_content}\n\nPlease answer questions based on this content."
    }
]

# Step 7: Interactive Q&A loop
print("✅ File processed. You can now ask questions about it. Type 'exit' to stop.")

while True:
    user_input = input("You: ")
    if user_input.strip().lower() == 'exit':
        break

    messages.append({"role": "user", "content": user_input})

    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=0.3
        )
        answer = response.choices[0].message.content
        print("Bot:", answer)
        messages.append({"role": "assistant", "content": answer})
    except Exception as e:
        print("❌ Error:", str(e))


Saving Resume template (1).pdf to Resume template (1) (1).pdf
✅ File processed. You can now ask questions about it. Type 'exit' to stop.
You: what is the name
Bot: The name mentioned in the document is Shreya Srivastava.
You: tell me the experience 
Bot: According to the document, Shreya Srivastava has the following work experience:

1. **Research Officer, Government of India (IIPS)** (Jun 2022 - Jul 2024)
   - Modeled child mortality across 600 villages using Small Area Estimation (SAE) and NFHS data.
   - Delivered insights that influenced public health policy decisions.
   - Built regression models and applied privacy-preserving analytics to health datasets.

2. **Sr. Executive, Data Science and Analytics, BARC INDIA** (Aug 2023 - Present)
   - Developed a Fitment Score model using MAD thresholds to flag low-engagement households, enhancing panel representativeness.
   - Automated panel analytics pipelines and predictive models, reducing reporting time by 40%.
   - Designed and ran 